In [1]:
import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd
import earthpy as et 
import shapely

In [2]:
# datasets: agricultura, mineria, manufactura y construccion

In [3]:
###############################################################################
###############################################################################


# AGRICULTURA

In [4]:
agricultura_data = '/home/horte/Documents/horte/Master/Machine Learning/Datasets/actividad economica/Agricultura/denue_00_11_shp/conjunto_de_datos'
mapa_agricultura = gpd.read_file(agricultura_data)

#remover columnas: 
#'nom_estab', 'raz_social', 'per_ocu', 'tipo_vial', 'nom_vial', 'tipo_v_e_1', 'nom_v_e_1','tipo_v_e_2', 'nom_v_e_2', 
#'tipo_v_e_3', 'nom_v_e_3', 'numero_ext','letra_ext', 'edificio', 'edificio_e', 'numero_int', 'letra_int','tipo_asent', 'nomb_asent', 
#'tipoCenCom', 'nom_CenCom', 'num_local','cod_postal', ageb', 'manzana', 'telefono', 'correoelec', 'www','tipoUniEco',

In [5]:
print(mapa_agricultura.columns)

Index(['id', 'clee', 'nom_estab', 'raz_social', 'codigo_act', 'nombre_act',
       'per_ocu', 'tipo_vial', 'nom_vial', 'tipo_v_e_1', 'nom_v_e_1',
       'tipo_v_e_2', 'nom_v_e_2', 'tipo_v_e_3', 'nom_v_e_3', 'numero_ext',
       'letra_ext', 'edificio', 'edificio_e', 'numero_int', 'letra_int',
       'tipo_asent', 'nomb_asent', 'tipoCenCom', 'nom_CenCom', 'num_local',
       'cod_postal', 'cve_ent', 'entidad', 'cve_mun', 'municipio', 'cve_loc',
       'localidad', 'ageb', 'manzana', 'telefono', 'correoelec', 'www',
       'tipoUniEco', 'latitud', 'longitud', 'fecha_alta', 'geometry'],
      dtype='object')


In [6]:
#ordenar por anio

# promedio de todos los datos y ponerlo en 2021
mapa_agricultura['anio'] = pd.to_datetime(mapa_agricultura['fecha_alta'], errors='coerce').dt.year
mapa_agricultura_2019 = mapa_agricultura[(mapa_agricultura['anio'] >= 2019) & (mapa_agricultura['anio'] <= 2023)]
mapa_agricultura_2019 = mapa_agricultura_2019.sort_values(by='anio', ascending=True)

In [7]:
# asignar clave geografica
mapa_agricultura_2019['CVE_GEO'] = mapa_agricultura_2019['cve_ent'].astype(str).str.zfill(2) + mapa_agricultura_2019['cve_mun'].astype(str).str.zfill(3)

#mover al inicio
mapa_agricultura_2019.insert(0, 'CVE_GEO', mapa_agricultura_2019.pop('CVE_GEO'))

In [8]:
mapa_agricultura_2019['per_ocu'].unique()

array(['0 a 5 personas', '11 a 30 personas', '6 a 10 personas',
       '51 a 100 personas', '31 a 50 personas', '101 a 250 personas',
       '251 y más personas'], dtype=object)

In [9]:
# rellenar con el mas alto

rango_max = {
    '0 a 5 personas': 5,
    '6 a 10 personas': 10,
    '11 a 30 personas': 30,
    '31 a 50 personas': 50,
    '51 a 100 personas': 100,
    '101 a 250 personas': 250,
    '251 y más personas': 500  
}

idx = mapa_agricultura_2019.columns.get_loc('per_ocu')
mapa_agricultura_2019.insert(idx + 1, 'per_ocu_max', mapa_agricultura_2019['per_ocu'].map(rango_max))

In [10]:

# removing columns
del_columns_agr = [
    'clee', 'id', 'codigo_act','nom_estab', 'raz_social', 'tipo_vial', 'nom_vial','tipo_v_e_1', 'nom_v_e_1', 'tipo_v_e_2', 'nom_v_e_2',
    'tipo_v_e_3', 'nom_v_e_3', 'numero_ext', 'letra_ext','edificio', 'edificio_e', 'numero_int', 'letra_int',
    'tipo_asent', 'nomb_asent', 'tipoCenCom', 'nom_CenCom','num_local', 'cod_postal', 'ageb', 'manzana', 'telefono',
    'correoelec', 'www', 'tipoUniEco', 'nombre_act','entidad','municipio','localidad']

mapa_agricultura_2019 = mapa_agricultura_2019.drop(columns=del_columns_agr)

In [11]:
# datos del 2019 - 2023
# sacar clave geografica
mapa_agricultura_2019

,CVE_GEO,per_ocu,per_ocu_max,cve_ent,cve_mun,cve_loc,latitud,longitud,fecha_alta,geometry,anio
0,27011,0 a 5 personas,5,27,011,0086,17.927039,-92.037636,2019-11,POINT (-92.03764 17.92704),2019.0
11852,30108,0 a 5 personas,5,30,108,0295,17.870216,-94.529265,2019-11,POINT (-94.52926 17.87022),2019.0
11839,30032,11 a 30 personas,30,30,032,0048,18.416347,-95.017902,2019-11,POINT (-95.0179 18.41635),2019.0
11769,30032,0 a 5 personas,5,30,032,0088,18.376712,-95.049724,2019-11,POINT (-95.04972 18.37671),2019.0
11770,30108,0 a 5 personas,5,30,108,0029,17.877924,-94.428807,2019-11,POINT (-94.42881 17.87792),2019.0
...,...,...,...,...,...,...,...,...,...,...,...
9818,24020,0 a 5 personas,5,24,020,0001,23.685031,-100.626648,2023-11,POINT (-100.62665 23.68503),2023.0
7845,22014,6 a 10 personas,10,22,014,0001,20.628169,-100.443075,2023-11,POINT (-100.44308 20.62817),2023.0
8111,14078,51 a 100 personas,100,14,078,0003,20.904246,-102.424585,2023-11,POINT (-102.42458 20.90425),2023.0
7710,26030,0 a 5 personas,5,26,030,0535,29.173824,-110.895717,2023-11,POINT (-110.89572 29.17382),2023.0


In [12]:
agricultura__mean = (mapa_agricultura_2019.groupby(['CVE_GEO', 'anio'])['per_ocu_max'].sum().reset_index(name='promedio_personas'))
agricultura__mean

,CVE_GEO,anio,promedio_personas
0,01001,2019.0,70
1,01001,2020.0,35
2,01003,2019.0,15
3,01005,2019.0,10
4,01005,2022.0,5
...,...,...,...
778,32047,2019.0,10
779,32047,2020.0,45
780,32052,2019.0,5
781,32055,2019.0,205


In [13]:
agricultura_pivot = agricultura__mean.pivot(index='CVE_GEO', columns='anio', values='promedio_personas')
agricultura_pivot

anio,2019.0,2020.0,2021.0,2022.0,2023.0
CVE_GEO,,,,,
01001,70.0,35.0,NaN,NaN,NaN
01003,15.0,NaN,NaN,NaN,NaN
01005,10.0,NaN,NaN,5.0,NaN
01007,15.0,NaN,NaN,NaN,NaN
02001,1070.0,190.0,500.0,NaN,70.0
...,...,...,...,...,...
32045,40.0,NaN,NaN,NaN,NaN
32046,40.0,NaN,NaN,NaN,NaN
32047,10.0,45.0,NaN,NaN,NaN


In [14]:
agricultura_pivot = agricultura_pivot.fillna(0)
agricultura_pivot

anio,2019.0,2020.0,2021.0,2022.0,2023.0
CVE_GEO,,,,,
01001,70.0,35.0,0.0,0.0,0.0
01003,15.0,0.0,0.0,0.0,0.0
01005,10.0,0.0,0.0,5.0,0.0
01007,15.0,0.0,0.0,0.0,0.0
02001,1070.0,190.0,500.0,0.0,70.0
...,...,...,...,...,...
32045,40.0,0.0,0.0,0.0,0.0
32046,40.0,0.0,0.0,0.0,0.0
32047,10.0,45.0,0.0,0.0,0.0


In [15]:
agricultura_pivot['avg_agricultura'] = agricultura_pivot.mean(axis=1)
agricultura_pivot['std_agricultura'] = agricultura_pivot.std(axis=1)
agricultura_pivot

anio,2019.0,2020.0,2021.0,2022.0,2023.0,avg_agricultura,std_agricultura
CVE_GEO,,,,,,,
01001,70.0,35.0,0.0,0.0,0.0,21.0,28.000000
01003,15.0,0.0,0.0,0.0,0.0,3.0,6.000000
01005,10.0,0.0,0.0,5.0,0.0,3.0,4.000000
01007,15.0,0.0,0.0,0.0,0.0,3.0,6.000000
02001,1070.0,190.0,500.0,0.0,70.0,366.0,391.438373
...,...,...,...,...,...,...,...
32045,40.0,0.0,0.0,0.0,0.0,8.0,16.000000
32046,40.0,0.0,0.0,0.0,0.0,8.0,16.000000
32047,10.0,45.0,0.0,0.0,0.0,11.0,17.435596


In [16]:
agricultura_2019_2023_mean = agricultura_pivot[['avg_agricultura','std_agricultura']].reset_index()
agricultura_2019_2023_mean

anio,CVE_GEO,avg_agricultura,std_agricultura
0,01001,21.0,28.000000
1,01003,3.0,6.000000
2,01005,3.0,4.000000
3,01007,3.0,6.000000
4,02001,366.0,391.438373
...,...,...,...
618,32045,8.0,16.000000
619,32046,8.0,16.000000
620,32047,11.0,17.435596
621,32052,1.0,2.000000


In [18]:
###############################################################################
###############################################################################


# MINERIA

In [19]:
mineria_data = '/home/horte/Documents/horte/Master/Machine Learning/Datasets/actividad economica/Mineria/denue_00_21_shp/conjunto_de_datos'

mapa_mineria = gpd.read_file(mineria_data)

In [20]:
print(mapa_mineria.columns)

Index(['id', 'clee', 'nom_estab', 'raz_social', 'codigo_act', 'nombre_act',
       'per_ocu', 'tipo_vial', 'nom_vial', 'tipo_v_e_1', 'nom_v_e_1',
       'tipo_v_e_2', 'nom_v_e_2', 'tipo_v_e_3', 'nom_v_e_3', 'numero_ext',
       'letra_ext', 'edificio', 'edificio_e', 'numero_int', 'letra_int',
       'tipo_asent', 'nomb_asent', 'tipoCenCom', 'nom_CenCom', 'num_local',
       'cod_postal', 'cve_ent', 'entidad', 'cve_mun', 'municipio', 'cve_loc',
       'localidad', 'ageb', 'manzana', 'telefono', 'correoelec', 'www',
       'tipoUniEco', 'latitud', 'longitud', 'fecha_alta', 'geometry'],
      dtype='object')


In [21]:
# sorting by year
mapa_mineria['anio'] = pd.to_datetime(mapa_mineria['fecha_alta'], errors='coerce').dt.year
mapa_mineria_2019 = mapa_mineria[(mapa_mineria['anio'] >= 2019) & (mapa_mineria['anio'] <= 2023)]
mapa_mineria_2019 = mapa_mineria_2019.sort_values(by='anio', ascending=True)


In [22]:
# asignar clave geografica
mapa_mineria_2019['CVE_GEO'] = mapa_mineria_2019['cve_ent'].astype(str).str.zfill(2) + mapa_mineria_2019['cve_mun'].astype(str).str.zfill(3)

#mover al inicio
mapa_mineria_2019.insert(0, 'CVE_GEO', mapa_mineria_2019.pop('CVE_GEO'))

In [23]:
mapa_mineria_2019['per_ocu'].unique()

array(['0 a 5 personas', '251 y más personas', '51 a 100 personas',
       '6 a 10 personas', '101 a 250 personas', '11 a 30 personas',
       '31 a 50 personas'], dtype=object)

In [24]:
# rellenar con el mas alto

rango_max = {
    '0 a 5 personas': 5,
    '6 a 10 personas': 10,
    '11 a 30 personas': 30,
    '31 a 50 personas': 50,
    '51 a 100 personas': 100,
    '101 a 250 personas': 250,
    '251 y más personas': 500  
}

idx = mapa_mineria_2019.columns.get_loc('per_ocu')
mapa_mineria_2019.insert(idx + 1, 'per_ocu_max', mapa_mineria_2019['per_ocu'].map(rango_max))

In [25]:
# removing columns
del_columns_min = [
    'clee', 'id', 'codigo_act','nom_estab', 'raz_social', 'tipo_vial', 'nom_vial','tipo_v_e_1', 'nom_v_e_1', 'tipo_v_e_2', 'nom_v_e_2',
    'tipo_v_e_3', 'nom_v_e_3', 'numero_ext', 'letra_ext','edificio', 'edificio_e', 'numero_int', 'letra_int',
    'tipo_asent', 'nomb_asent', 'tipoCenCom', 'nom_CenCom','num_local', 'cod_postal', 'ageb', 'manzana', 'telefono',
    'correoelec', 'www', 'tipoUniEco', 'nombre_act','entidad','municipio','localidad']

mapa_mineria_2019 = mapa_mineria_2019.drop(columns=del_columns_agr)

In [26]:
mapa_mineria_2019

,CVE_GEO,per_ocu,per_ocu_max,cve_ent,cve_mun,cve_loc,latitud,longitud,fecha_alta,geometry,anio
3405,14002,0 a 5 personas,5,14,002,0020,20.362899,-103.560039,2019-11,POINT (-103.56004 20.3629),2019.0
174,26030,0 a 5 personas,5,26,030,0001,29.127162,-110.951271,2019-04,POINT (-110.95127 29.12716),2019.0
177,08051,251 y más personas,500,08,051,0350,28.192050,-108.374820,2019-04,POINT (-108.37482 28.19205),2019.0
190,09010,51 a 100 personas,100,09,010,0001,19.367027,-99.259150,2019-04,POINT (-99.25915 19.36703),2019.0
191,09016,6 a 10 personas,10,09,016,0001,19.434104,-99.212811,2019-04,POINT (-99.21281 19.4341),2019.0
...,...,...,...,...,...,...,...,...,...,...,...
679,10004,11 a 30 personas,30,10,004,0015,25.095780,-103.770062,2023-11,POINT (-103.77006 25.09578),2023.0
2450,30131,51 a 100 personas,100,30,131,0001,20.525828,-97.458902,2023-11,POINT (-97.4589 20.52583),2023.0
2445,30100,0 a 5 personas,5,30,100,0050,19.139223,-96.389879,2023-11,POINT (-96.38988 19.13922),2023.0
973,10005,51 a 100 personas,100,10,005,0001,24.029897,-104.648206,2023-11,POINT (-104.64821 24.0299),2023.0


In [27]:
mineria__mean = (mapa_mineria_2019.groupby(['CVE_GEO', 'anio'])['per_ocu_max'].sum().reset_index(name='promedio_personas'))
mineria__mean

,CVE_GEO,anio,promedio_personas
0,01001,2020.0,10
1,01003,2020.0,5
2,01005,2019.0,500
3,01005,2020.0,30
4,01009,2020.0,5
...,...,...,...
426,32049,2020.0,5
427,32055,2020.0,15
428,32056,2019.0,500
429,32056,2020.0,5


In [28]:
mineria__mean = mineria__mean.pivot(index='CVE_GEO', columns='anio', values='promedio_personas')
mineria__mean

anio,2019.0,2020.0,2021.0,2022.0,2023.0
CVE_GEO,,,,,
01001,NaN,10.0,NaN,NaN,NaN
01003,NaN,5.0,NaN,NaN,NaN
01005,500.0,30.0,NaN,NaN,NaN
01009,NaN,5.0,NaN,NaN,NaN
02001,5.0,5.0,NaN,NaN,NaN
...,...,...,...,...,...
32038,NaN,5.0,NaN,NaN,NaN
32039,NaN,110.0,NaN,NaN,NaN
32049,NaN,5.0,NaN,NaN,NaN


In [29]:
mineria__mean = mineria__mean.fillna(0)
mineria__mean

anio,2019.0,2020.0,2021.0,2022.0,2023.0
CVE_GEO,,,,,
01001,0.0,10.0,0.0,0.0,0.0
01003,0.0,5.0,0.0,0.0,0.0
01005,500.0,30.0,0.0,0.0,0.0
01009,0.0,5.0,0.0,0.0,0.0
02001,5.0,5.0,0.0,0.0,0.0
...,...,...,...,...,...
32038,0.0,5.0,0.0,0.0,0.0
32039,0.0,110.0,0.0,0.0,0.0
32049,0.0,5.0,0.0,0.0,0.0


In [30]:
mineria__mean['avg_mineria'] = mineria__mean.mean(axis=1)
mineria__mean['std_mineria'] = mineria__mean.std(axis=1)
mineria__mean

anio,2019.0,2020.0,2021.0,2022.0,2023.0,avg_mineria,std_mineria
CVE_GEO,,,,,,,
01001,0.0,10.0,0.0,0.0,0.0,2.0,4.000000
01003,0.0,5.0,0.0,0.0,0.0,1.0,2.000000
01005,500.0,30.0,0.0,0.0,0.0,106.0,197.342342
01009,0.0,5.0,0.0,0.0,0.0,1.0,2.000000
02001,5.0,5.0,0.0,0.0,0.0,2.0,2.449490
...,...,...,...,...,...,...,...
32038,0.0,5.0,0.0,0.0,0.0,1.0,2.000000
32039,0.0,110.0,0.0,0.0,0.0,22.0,44.000000
32049,0.0,5.0,0.0,0.0,0.0,1.0,2.000000


In [31]:
mineria_2019_2023_mean = mineria__mean[['avg_mineria','std_mineria']].reset_index()
mineria_2019_2023_mean

anio,CVE_GEO,avg_mineria,std_mineria
0,01001,2.0,4.000000
1,01003,1.0,2.000000
2,01005,106.0,197.342342
3,01009,1.0,2.000000
4,02001,2.0,2.449490
...,...,...,...
317,32038,1.0,2.000000
318,32039,22.0,44.000000
319,32049,1.0,2.000000
320,32055,3.0,6.000000


In [32]:
###############################################################################
###############################################################################


# INDUSTRIA MANUFACTURERA


In [33]:
manufactura_path = '/home/horte/Documents/horte/Master/Machine Learning/Datasets/actividad economica/Ind. Manufacturera/denue_00_31-33_shp/conjunto_de_datos'
mapa_manufactura = gpd.read_file(manufactura_path)

In [34]:
# sorting by year
mapa_manufactura['anio'] = pd.to_datetime(mapa_manufactura['fecha_alta'], errors='coerce').dt.year
mapa_manufactura_2019 = mapa_manufactura[(mapa_manufactura['anio'] >= 2019) & (mapa_manufactura['anio'] <= 2023)]
mapa_manufactura_2019 = mapa_manufactura_2019.sort_values(by='anio', ascending=True)


In [35]:
# asignar clave geografica
mapa_manufactura_2019['CVE_GEO'] = mapa_manufactura_2019['cve_ent'].astype(str).str.zfill(2) + mapa_manufactura_2019['cve_mun'].astype(str).str.zfill(3)

#mover al inicio
mapa_manufactura_2019.insert(0, 'CVE_GEO', mapa_manufactura_2019.pop('CVE_GEO'))

In [36]:
# rellenar con el mas alto

rango_max = {
    '0 a 5 personas': 5,
    '6 a 10 personas': 10,
    '11 a 30 personas': 30,
    '31 a 50 personas': 50,
    '51 a 100 personas': 100,
    '101 a 250 personas': 250,
    '251 y más personas': 500  
}

idx = mapa_manufactura_2019.columns.get_loc('per_ocu')
mapa_manufactura_2019.insert(idx + 1, 'per_ocu_max', mapa_manufactura_2019['per_ocu'].map(rango_max))

In [37]:
# removing columns
del_columns_man = [
    'clee', 'id', 'codigo_act','nom_estab', 'raz_social', 'tipo_vial', 'nom_vial','tipo_v_e_1', 'nom_v_e_1', 'tipo_v_e_2', 'nom_v_e_2',
    'tipo_v_e_3', 'nom_v_e_3', 'numero_ext', 'letra_ext','edificio', 'edificio_e', 'numero_int', 'letra_int',
    'tipo_asent', 'nomb_asent', 'tipoCenCom', 'nom_CenCom','num_local', 'cod_postal', 'ageb', 'manzana', 'telefono',
    'correoelec', 'www', 'tipoUniEco', 'nombre_act','entidad','municipio','localidad']

mapa_manufactura_2019 = mapa_manufactura_2019.drop(columns=del_columns_agr)

In [38]:
mapa_manufactura_2019

,CVE_GEO,per_ocu,per_ocu_max,cve_ent,cve_mun,cve_loc,latitud,longitud,fecha_alta,geometry,anio
423745,20513,0 a 5 personas,5,20,513,0001,16.817667,-95.136860,2019-11,POINT (-95.13686 16.81767),2019.0
423710,20390,0 a 5 personas,5,20,390,0001,17.052223,-96.706044,2019-11,POINT (-96.70604 17.05222),2019.0
423713,20091,0 a 5 personas,5,20,091,0001,17.102834,-96.663662,2019-11,POINT (-96.66366 17.10283),2019.0
423716,20565,0 a 5 personas,5,20,565,0006,16.929372,-96.686660,2019-11,POINT (-96.68666 16.92937),2019.0
423717,20039,0 a 5 personas,5,20,039,0001,17.810408,-97.783064,2019-11,POINT (-97.78306 17.81041),2019.0
...,...,...,...,...,...,...,...,...,...,...,...
137789,11020,31 a 50 personas,50,11,020,0001,21.080847,-101.672067,2023-11,POINT (-101.67207 21.08085),2023.0
137780,11020,11 a 30 personas,30,11,020,0001,21.088816,-101.593625,2023-11,POINT (-101.59363 21.08882),2023.0
505066,22011,11 a 30 personas,30,22,011,0504,20.571132,-100.256853,2023-11,POINT (-100.25685 20.57113),2023.0
137751,11037,31 a 50 personas,50,11,037,0699,21.003427,-101.479529,2023-11,POINT (-101.47953 21.00343),2023.0


In [39]:
manufactura__mean = (mapa_manufactura_2019.groupby(['CVE_GEO', 'anio'])['per_ocu_max'].mean().reset_index(name='promedio_personas'))
manufactura__mean

,CVE_GEO,anio,promedio_personas
0,01001,2019.0,12.074115
1,01001,2020.0,62.692308
2,01001,2021.0,23.076923
3,01001,2022.0,54.000000
4,01001,2023.0,73.055556
...,...,...,...
4175,32056,2020.0,15.000000
4176,32056,2021.0,5.000000
4177,32056,2023.0,22.500000
4178,32057,2019.0,5.500000


In [40]:
manufactura__mean = manufactura__mean.pivot(index='CVE_GEO', columns='anio', values='promedio_personas')
manufactura__mean

anio,2019.0,2020.0,2021.0,2022.0,2023.0
CVE_GEO,,,,,
01001,12.074115,62.692308,23.076923,54.00,73.055556
01002,5.416667,NaN,5.000000,NaN,NaN
01003,5.606061,NaN,8.571429,NaN,NaN
01004,5.000000,NaN,NaN,NaN,NaN
01005,24.361111,138.750000,8.750000,151.25,80.000000
...,...,...,...,...,...
32053,5.000000,NaN,NaN,NaN,NaN
32054,5.000000,5.000000,NaN,NaN,NaN
32055,5.384615,NaN,5.000000,NaN,5.000000


In [41]:
manufactura__mean = manufactura__mean.fillna(0)
manufactura__mean

anio,2019.0,2020.0,2021.0,2022.0,2023.0
CVE_GEO,,,,,
01001,12.074115,62.692308,23.076923,54.00,73.055556
01002,5.416667,0.000000,5.000000,0.00,0.000000
01003,5.606061,0.000000,8.571429,0.00,0.000000
01004,5.000000,0.000000,0.000000,0.00,0.000000
01005,24.361111,138.750000,8.750000,151.25,80.000000
...,...,...,...,...,...
32053,5.000000,0.000000,0.000000,0.00,0.000000
32054,5.000000,5.000000,0.000000,0.00,0.000000
32055,5.384615,0.000000,5.000000,0.00,5.000000


In [42]:
manufactura__mean['avg_manufactura'] = manufactura__mean.mean(axis=1)
manufactura__mean['std_manufactura'] = manufactura__mean.std(axis=1)
manufactura__mean

anio,2019.0,2020.0,2021.0,2022.0,2023.0,avg_manufactura,std_manufactura
CVE_GEO,,,,,,,
01001,12.074115,62.692308,23.076923,54.00,73.055556,44.979780,23.434441
01002,5.416667,0.000000,5.000000,0.00,0.000000,2.083333,2.554952
01003,5.606061,0.000000,8.571429,0.00,0.000000,2.835498,3.597140
01004,5.000000,0.000000,0.000000,0.00,0.000000,1.000000,2.000000
01005,24.361111,138.750000,8.750000,151.25,80.000000,80.622222,57.790030
...,...,...,...,...,...,...,...
32053,5.000000,0.000000,0.000000,0.00,0.000000,1.000000,2.000000
32054,5.000000,5.000000,0.000000,0.00,0.000000,2.000000,2.449490
32055,5.384615,0.000000,5.000000,0.00,5.000000,3.076923,2.516220


In [43]:
manufactura_2019_2023_mean = manufactura__mean[['avg_manufactura','std_manufactura']].reset_index()
manufactura_2019_2023_mean

anio,CVE_GEO,avg_manufactura,std_manufactura
0,01001,44.979780,23.434441
1,01002,2.083333,2.554952
2,01003,2.835498,3.597140
3,01004,1.000000,2.000000
4,01005,80.622222,57.790030
...,...,...,...
2335,32053,1.000000,2.000000
2336,32054,2.000000,2.449490
2337,32055,3.076923,2.516220
2338,32056,9.888060,7.944169


In [44]:
###############################################################################
###############################################################################


# CCONSTRUCCION


In [45]:
construccion_path = '/home/horte/Documents/horte/Master/Machine Learning/Datasets/actividad economica/Construccion/denue_00_23_shp/conjunto_de_datos'
mapa_construccion = gpd.read_file(construccion_path)

In [46]:
# source by year
mapa_construccion['anio'] = pd.to_datetime(mapa_construccion['fecha_alta'], errors='coerce').dt.year
mapa_construccion_2019 = mapa_construccion[(mapa_construccion['anio'] >= 2019) & (mapa_construccion['anio'] <= 2023)]
mapa_construccion_2019 = mapa_construccion_2019.sort_values(by='anio', ascending=True)

In [47]:
# asignar clave geografica
mapa_construccion_2019['CVE_GEO'] = mapa_construccion_2019['cve_ent'].astype(str).str.zfill(2) + mapa_construccion_2019['cve_mun'].astype(str).str.zfill(3)

#mover al inicio
mapa_construccion_2019.insert(0, 'CVE_GEO', mapa_construccion_2019.pop('CVE_GEO'))

In [48]:
# rellenar con el mas alto

rango_max = {
    '0 a 5 personas': 5,
    '6 a 10 personas': 10,
    '11 a 30 personas': 30,
    '31 a 50 personas': 50,
    '51 a 100 personas': 100,
    '101 a 250 personas': 250,
    '251 y más personas': 500  
}

idx = mapa_construccion_2019.columns.get_loc('per_ocu')
mapa_construccion_2019.insert(idx + 1, 'per_ocu_max', mapa_construccion_2019['per_ocu'].map(rango_max))

In [49]:
# removing columns
del_columns_man = [
    'clee', 'id', 'codigo_act','nom_estab', 'raz_social', 'tipo_vial', 'nom_vial','tipo_v_e_1', 'nom_v_e_1', 'tipo_v_e_2', 'nom_v_e_2',
    'tipo_v_e_3', 'nom_v_e_3', 'numero_ext', 'letra_ext','edificio', 'edificio_e', 'numero_int', 'letra_int',
    'tipo_asent', 'nomb_asent', 'tipoCenCom', 'nom_CenCom','num_local', 'cod_postal', 'ageb', 'manzana', 'telefono',
    'correoelec', 'www', 'tipoUniEco', 'nombre_act','entidad','municipio','localidad']

mapa_construccion_2019 = mapa_construccion_2019.drop(columns=del_columns_agr)

In [50]:
mapa_construccion_2019

,CVE_GEO,per_ocu,per_ocu_max,cve_ent,cve_mun,cve_loc,latitud,longitud,fecha_alta,geometry,anio
8264,02004,0 a 5 personas,5,02,004,0001,32.485323,-116.922710,2019-11,POINT (-116.92271 32.48532),2019.0
8260,02001,11 a 30 personas,30,02,001,0001,31.873054,-116.576320,2019-11,POINT (-116.57632 31.87305),2019.0
8254,01005,0 a 5 personas,5,01,005,0001,21.948859,-102.312470,2019-11,POINT (-102.31247 21.94886),2019.0
8243,01001,31 a 50 personas,50,01,001,0001,21.915873,-102.306766,2019-11,POINT (-102.30677 21.91587),2019.0
8235,02004,0 a 5 personas,5,02,004,0001,32.438419,-116.971153,2019-11,POINT (-116.97115 32.43842),2019.0
...,...,...,...,...,...,...,...,...,...,...,...
643,04009,11 a 30 personas,30,04,009,0001,18.619269,-90.742026,2023-11,POINT (-90.74203 18.61927),2023.0
14478,17029,0 a 5 personas,5,17,029,0001,18.872328,-99.062890,2023-11,POINT (-99.06289 18.87233),2023.0
1414,11017,11 a 30 personas,30,11,017,0001,20.671435,-101.350425,2023-11,POINT (-101.35043 20.67143),2023.0
6706,08017,101 a 250 personas,250,08,017,0001,28.413255,-106.857354,2023-11,POINT (-106.85735 28.41325),2023.0


In [51]:
construccion__mean = (mapa_construccion_2019.groupby(['CVE_GEO', 'anio'])['per_ocu_max'].mean().reset_index(name='promedio_personas'))
construccion__mean

,CVE_GEO,anio,promedio_personas
0,01001,2019.0,31.071429
1,01001,2020.0,38.262712
2,01001,2021.0,36.250000
3,01001,2023.0,15.384615
4,01002,2020.0,17.500000
...,...,...,...
1306,32048,2019.0,10.000000
1307,32048,2020.0,30.000000
1308,32056,2019.0,7.500000
1309,32056,2020.0,30.277778


In [52]:
construccion__mean = construccion__mean.pivot(index='CVE_GEO', columns='anio', values='promedio_personas')
construccion__mean

anio,2019.0,2020.0,2021.0,2022.0,2023.0
CVE_GEO,,,,,
01001,31.071429,38.262712,36.25,NaN,15.384615
01002,NaN,17.500000,5.00,NaN,NaN
01003,NaN,11.666667,NaN,NaN,NaN
01005,33.750000,31.818182,NaN,NaN,50.000000
01006,NaN,30.000000,NaN,NaN,NaN
...,...,...,...,...,...
32038,NaN,10.000000,NaN,NaN,NaN
32039,NaN,15.000000,NaN,NaN,NaN
32042,5.000000,27.142857,NaN,NaN,5.000000


In [53]:
construccion__mean = construccion__mean.fillna(0)
construccion__mean

anio,2019.0,2020.0,2021.0,2022.0,2023.0
CVE_GEO,,,,,
01001,31.071429,38.262712,36.25,0.0,15.384615
01002,0.000000,17.500000,5.00,0.0,0.000000
01003,0.000000,11.666667,0.00,0.0,0.000000
01005,33.750000,31.818182,0.00,0.0,50.000000
01006,0.000000,30.000000,0.00,0.0,0.000000
...,...,...,...,...,...
32038,0.000000,10.000000,0.00,0.0,0.000000
32039,0.000000,15.000000,0.00,0.0,0.000000
32042,5.000000,27.142857,0.00,0.0,5.000000


In [54]:
construccion__mean['avg_construccion'] = construccion__mean.mean(axis=1)
construccion__mean['std_construccion'] = construccion__mean.std(axis=1)
construccion__mean

anio,2019.0,2020.0,2021.0,2022.0,2023.0,avg_construccion,std_construccion
CVE_GEO,,,,,,,
01001,31.071429,38.262712,36.25,0.0,15.384615,24.193751,14.515714
01002,0.000000,17.500000,5.00,0.0,0.000000,4.500000,6.782330
01003,0.000000,11.666667,0.00,0.0,0.000000,2.333333,4.666667
01005,33.750000,31.818182,0.00,0.0,50.000000,23.113636,19.901047
01006,0.000000,30.000000,0.00,0.0,0.000000,6.000000,12.000000
...,...,...,...,...,...,...,...
32038,0.000000,10.000000,0.00,0.0,0.000000,2.000000,4.000000
32039,0.000000,15.000000,0.00,0.0,0.000000,3.000000,6.000000
32042,5.000000,27.142857,0.00,0.0,5.000000,7.428571,10.107585


In [55]:
construccion_2019_2023_mean = construccion__mean[['avg_construccion','std_construccion']].reset_index()
construccion_2019_2023_mean

anio,CVE_GEO,avg_construccion,std_construccion
0,01001,24.193751,14.515714
1,01002,4.500000,6.782330
2,01003,2.333333,4.666667
3,01005,23.113636,19.901047
4,01006,6.000000,12.000000
...,...,...,...
693,32038,2.000000,4.000000
694,32039,3.000000,6.000000
695,32042,7.428571,10.107585
696,32048,8.000000,11.661904


In [232]:
#####
# MERGE DATA FRAMES
#####

In [60]:
all_data = pd.merge(
    agricultura_2019_2023_mean[['CVE_GEO', 'avg_agricultura', 'std_agricultura']],
    construccion_2019_2023_mean[['CVE_GEO', 'avg_construccion', 'std_construccion']],
    on='CVE_GEO',
    how='outer'  
)

In [61]:
all_data

anio,CVE_GEO,avg_agricultura,std_agricultura,avg_construccion,std_construccion
0,01001,21.0,28.000000,24.193751,14.515714
1,01002,NaN,NaN,4.500000,6.782330
2,01003,3.0,6.000000,2.333333,4.666667
3,01005,3.0,4.000000,23.113636,19.901047
4,01006,NaN,NaN,6.000000,12.000000
...,...,...,...,...,...
1013,32047,11.0,17.435596,NaN,NaN
1014,32048,NaN,NaN,8.000000,11.661904
1015,32052,1.0,2.000000,NaN,NaN
1016,32055,46.0,80.087452,NaN,NaN


In [63]:
all_data = pd.merge(
    all_data,
    manufactura_2019_2023_mean[['CVE_GEO', 'avg_manufactura', 'std_manufactura']],
    on='CVE_GEO',
    how='outer'  
)

In [64]:
all_data

anio,CVE_GEO,avg_agricultura,std_agricultura,avg_construccion,std_construccion,avg_manufactura,std_manufactura
0,01001,21.0,28.000000,24.193751,14.515714,44.979780,23.434441
1,01002,NaN,NaN,4.500000,6.782330,2.083333,2.554952
2,01003,3.0,6.000000,2.333333,4.666667,2.835498,3.597140
3,01004,NaN,NaN,NaN,NaN,1.000000,2.000000
4,01005,3.0,4.000000,23.113636,19.901047,80.622222,57.790030
...,...,...,...,...,...,...,...
2345,32053,NaN,NaN,NaN,NaN,1.000000,2.000000
2346,32054,NaN,NaN,NaN,NaN,2.000000,2.449490
2347,32055,46.0,80.087452,NaN,NaN,3.076923,2.516220
2348,32056,NaN,NaN,13.555556,13.814691,9.888060,7.944169


In [65]:
all_data = pd.merge(
    all_data,
    mineria_2019_2023_mean[['CVE_GEO', 'avg_mineria', 'std_mineria']],
    on='CVE_GEO',
    how='outer'  
)

In [66]:
all_data

anio,CVE_GEO,avg_agricultura,std_agricultura,avg_construccion,std_construccion,avg_manufactura,std_manufactura,avg_mineria,std_mineria
0,01001,21.0,28.000000,24.193751,14.515714,44.979780,23.434441,2.0,4.000000
1,01002,NaN,NaN,4.500000,6.782330,2.083333,2.554952,NaN,NaN
2,01003,3.0,6.000000,2.333333,4.666667,2.835498,3.597140,1.0,2.000000
3,01004,NaN,NaN,NaN,NaN,1.000000,2.000000,NaN,NaN
4,01005,3.0,4.000000,23.113636,19.901047,80.622222,57.790030,106.0,197.342342
...,...,...,...,...,...,...,...,...,...
2353,32053,NaN,NaN,NaN,NaN,1.000000,2.000000,NaN,NaN
2354,32054,NaN,NaN,NaN,NaN,2.000000,2.449490,NaN,NaN
2355,32055,46.0,80.087452,NaN,NaN,3.076923,2.516220,3.0,6.000000
2356,32056,NaN,NaN,13.555556,13.814691,9.888060,7.944169,191.0,232.430635


In [69]:
all_data.to_csv('all_data_horte.csv', index=False)